<a href="https://colab.research.google.com/github/samitha278/miniVGG/blob/main/build_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Simple Convolution function

In [2]:
def Convolution(image,kernel):

  x,y = image.shape

  a,b = kernel.shape

  feature_map = []

  for i in range(y-b):

    row = []

    for j in range(x-a):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [3]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel)


In [4]:
feature_map.shape

torch.Size([29, 29])

## Convolution with stride

In [5]:
def Convolution(image,kernel,stride=(1,1)):

  x,y = image.shape
  a,b = kernel.shape

  r,c = stride

  feature_map = []

  for i in range(0,y-b,c):

    row = []

    for j in range(0,x-a,r):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [6]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel,(2,1))      # 2: aloge side rows , 1: along side columns


In [7]:
feature_map.shape

torch.Size([29, 15])

## Adding Padding

In [8]:
def add_padding(image, padding):



  r,c = padding

  if r>0 :
    rows = torch.zeros((r,image.shape[1]))

    image = torch.cat((rows , image , rows),dim=0)

  if c>0:

    columns = torch.zeros((c,image.shape[0]))

    image = torch.cat((columns , image.T , columns),dim=0)


  return image.T



In [9]:
add_padding(torch.randn(2,2),(2,3))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.0341,  0.1381,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.8526,  0.5391,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [10]:
def Convolution(image,kernel,stride=(1,1),padding = (0,0)):


  a,b = kernel.shape

  r,c = stride

  image_pd = image if padding==(0,0) else add_padding(image,padding)

  feature_map = []

  x,y = image_pd.shape

  for i in range(0,y-b,c):

    row = []

    for j in range(0,x-a,r):

      temp = (image_pd[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [11]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel, padding=(2,2))


In [12]:
feature_map.shape

torch.Size([33, 33])

### Simple adding pad func

In [13]:
def add_padding2(matrix,padding):

  n,m = matrix.shape

  r,c = padding

  padded_matrix = torch.zeros((n+r*2,m+c*2))

  padded_matrix[r:r+n,c:c+m] = matrix

  return padded_matrix

In [14]:
add_padding2(torch.randn(2,2),(2,3))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.5153,  0.4146,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -1.0275,  0.0473,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

### Adding Dialation

In [15]:
def Convolution(image,kernel,stride=(1,1),padding = (0,0),dilation=(1,1)):


  a,b = kernel.shape

  r,c = stride

  d1,d2 = dilation

  image_pd = image if padding==(0,0) else add_padding2(image,padding)

  feature_map = []

  x,y = image_pd.shape


  for i in range(0,x-a*d1+1,r):

    row = []

    for j in range(0,y-b*d2+1,c):

      temp = (image_pd[i:i+a*d1:d1,j:j+b*d2:d2] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [16]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel,(2,2),(2,2) ,dilation=(2,2))

In [17]:
feature_map.shape

torch.Size([16, 16])

### Test : comparision with pytorch conv2d

In [18]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)

out_custom = Convolution(image, kernel, stride=(2,2), padding=(1,1), dilation=(2,2))


img4d = image.unsqueeze(0).unsqueeze(0)       # (1,1,4,4)
kernel4d = kernel.unsqueeze(0).unsqueeze(0) # (1,1,2,2)

out_torch = F.conv2d(img4d, kernel4d, stride=(2,2), padding=(1,1), dilation=(2,2))

print("Custom:\n", out_custom.shape)
print("Torch:\n", out_torch.squeeze().shape)
print("Match? ->", torch.allclose(out_custom,out_torch.squeeze()))

Custom:
 torch.Size([15, 15])
Torch:
 torch.Size([15, 15])
Match? -> True


# All at once

In [19]:
def add_padding2(matrix,padding):
  n,m = matrix.shape
  r,c = padding

  padded_matrix = torch.zeros((n+r*2,m+c*2))
  padded_matrix[r:r+n,c:c+m] = matrix

  return padded_matrix

In [20]:
def Convolution(image, kernel, stride=(1,1), padding=(0,0), dilation=(1,1)):
    a, b = kernel.shape
    r, c = stride
    d1, d2 = dilation

    image_pd = image if padding == (0,0) else add_padding2(image, padding)
    feature_map = []
    x, y = image_pd.shape

    for i in range(0, x - a*d1 + 1, r):
        row = []
        for j in range(0, y - b*d2 + 1, c):

            temp = (image_pd[i:i+a*d1:d1, j:j+b*d2:d2] * kernel).sum(dim=(0,1))
            row.append(temp.item())
        feature_map.append(row)

    return torch.tensor(feature_map)

In [21]:
#torch.manual_seed(220064)
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)

out_custom = Convolution(image, kernel, stride=(2,2), padding=(1,1), dilation=(2,2))


img4d = image.unsqueeze(0).unsqueeze(0)       # (1,1,4,4)
kernel4d = kernel.unsqueeze(0).unsqueeze(0) # (1,1,2,2)

out_torch = F.conv2d(img4d, kernel4d, stride=(2,2), padding=(1,1), dilation=(2,2))

print("Custom:\n", out_custom.shape)
print("Torch:\n", out_torch.squeeze().shape)
print("Match? ->", torch.allclose(out_custom,out_torch.squeeze()))

Custom:
 torch.Size([15, 15])
Torch:
 torch.Size([15, 15])
Match? -> True


### Test nn.Conv2d

In [22]:
torch.manual_seed(200)

c_l = nn.Conv2d(2,4,3)      # in channel = 2, out channel = 4, kernel size = 3x3


sd = c_l.__dict__['_parameters']
wei = sd['weight']
bias = sd['bias']

img = torch.randn(2,12,12)


In [23]:
# for outout channel 1
out = Convolution(img[0],wei[0,0]) + Convolution(img[1],wei[0,1]) + bias[0]

In [24]:
out_real = c_l(img)[0]

In [25]:
torch.allclose(out_real,out)

True

In [26]:
out

tensor([[-1.1716, -0.8489,  0.0087,  0.3687,  0.0612, -0.4664, -0.0916,  0.3448,
         -0.3278,  0.4495],
        [-0.9561,  0.1782,  0.3831, -0.2250, -0.4388, -0.5568, -0.3419,  0.2183,
          0.0076, -0.6042],
        [ 0.0471, -0.4671, -0.2238, -0.1945,  0.2181, -0.4026, -0.5394, -0.6788,
         -0.5775, -0.8300],
        [-0.1201,  0.0371, -0.3132, -0.2208, -0.2247, -0.3121,  0.1181, -0.3183,
          0.2815, -1.5969],
        [ 0.3042,  0.3495, -0.1749,  0.1566, -1.6928, -0.1662,  0.3522, -0.1735,
         -0.7710, -0.8561],
        [ 0.1167,  0.8531,  0.7492,  0.0081, -0.7360, -0.0125, -0.1202, -1.4806,
          0.8252,  0.3005],
        [-0.3631, -0.4048,  1.2756,  0.3087, -1.7028,  0.7630,  0.2855, -1.1258,
          0.5164,  0.3389],
        [-0.2170,  0.4665, -0.0310,  0.3007,  0.1645, -0.0165, -0.6288, -0.9614,
         -0.2822,  0.3014],
        [-1.0514, -0.2228, -0.3419,  0.8720, -0.4499,  0.1352, -0.1190, -0.4014,
          0.8269,  0.5460],
        [ 0.5875,  

In [35]:
wei[0,0].shape, bias

(torch.Size([3, 3]),
 Parameter containing:
 tensor([-0.2148,  0.1413, -0.0320,  0.0114], requires_grad=True))

In [38]:
Convolution(img[0],wei[0,0]) + Convolution(img[1],wei[0,1]) #+ torch.tensor([-0.2148])  # bias braodcasting

tensor([[-0.9568, -0.6341,  0.2235,  0.5836,  0.2760, -0.2516,  0.1232,  0.5596,
         -0.1130,  0.6643],
        [-0.7413,  0.3930,  0.5980, -0.0102, -0.2240, -0.3420, -0.1271,  0.4332,
          0.2224, -0.3893],
        [ 0.2619, -0.2522, -0.0090,  0.0204,  0.4329, -0.1878, -0.3246, -0.4639,
         -0.3626, -0.6152],
        [ 0.0947,  0.2519, -0.0984, -0.0059, -0.0099, -0.0973,  0.3329, -0.1035,
          0.4963, -1.3820],
        [ 0.5190,  0.5643,  0.0400,  0.3714, -1.4779,  0.0486,  0.5670,  0.0413,
         -0.5561, -0.6412],
        [ 0.3316,  1.0679,  0.9641,  0.2230, -0.5212,  0.2023,  0.0947, -1.2658,
          1.0401,  0.5153],
        [-0.1482, -0.1899,  1.4905,  0.5235, -1.4880,  0.9779,  0.5003, -0.9110,
          0.7312,  0.5538],
        [-0.0021,  0.6814,  0.1839,  0.5156,  0.3793,  0.1983, -0.4140, -0.7465,
         -0.0674,  0.5162],
        [-0.8365, -0.0079, -0.1271,  1.0868, -0.2350,  0.3500,  0.0959, -0.1865,
          1.0418,  0.7609],
        [ 0.8024,  

## Conv2d Class

In [28]:
class Conv2d(nn.Module):

  def __init__(self,
               in_channels, out_channels, kernel_size,
               stride = 1, padding = 0, dilation= 1,
               bias = True
               ):
    super().__init__()

    assert kernel_size%2==1 , "kernel size must be odd"

    self.weights = torch.randn((out_channels,in_channels,kernel_size,kernel_size)) * (in_channels**-0.5)
    self.bias = torch.randn(out_channels) if bias else None



    #variables
    self.stride = stride
    self.padding = padding
    self.dilation = dilation



  def forward(self,x):

    B,C,H,W = x.shape

    H_out= W_out = ((H+2*self.padding-(self.dilation*self.kernel_size-1)-1)//self.stride)+1    # calculate output size

    x = x if self.padding == 0 else x.add_padding(self.padding)     #adding padding

    self.feature_map = torch.zeros((B,self.out_channels,H_out,W_out))    #output feature map

    for b in range(B):
      for out_c in range(self.out_channels):


        #Convolution
        for h in range(H_out):
          for w in range(W_out):

            #feature_map[b,out_c,h,w] = (x[i:i+a*d1:d1, j:j+b*d2:d2] * self.weight[out_c]).sum(dim=(0,1))

            pass





    return self.feature_map












  def add_padding(self,padding):
    B,C,H,W = self.shape
    p = padding

    padded_x = torch.zeros((B,C,H+p*2,W+p*2))
    padded_x[:,:,p:p+H,p:p+W] = self

    return padded_x





In [31]:
# nn.Conv2d()

"""
in_channels: int, out_channels: int, kernel_size: _size_2_t,
stride: _size_2_t = 1, padding: _size_2_t | str = 0, dilation: _size_2_t = 1,
groups: int = 1, bias: bool = True, padding_mode: str = "zeros",
device: Any | None = None, dtype: Any | None = None) -> None
"""

'\nin_channels: int, out_channels: int, kernel_size: _size_2_t, \nstride: _size_2_t = 1, padding: _size_2_t | str = 0, dilation: _size_2_t = 1, \ngroups: int = 1, bias: bool = True, padding_mode: str = "zeros", \ndevice: Any | None = None, dtype: Any | None = None) -> None\n'